## Advent of Code 2023 day 2 🟥 🟩 🟦 

This is my first time trying out the [FParsec](https://github.com/stephan-tolksdorf/fparsec) library.

For me the most painful/tedious part of Advent of Code is parsing the input data with a collection of `String.Split`, `String.Trim` and various array operations.

FParsec is far nicer to work with and I think I'll be using it from now on.

In [114]:
#r "nuget:FParsec"

open FParsec

let loadFile fname = System.IO.File.ReadAllLines(fname)

let sample = "day2.sample.txt"
let data = "day2.data.txt"

Installed Packages FParsec, 1.1.1

In [115]:
let str = pstring
let gameNumberParser = str "Game " >>. pint32 .>> str ": " // "Game (n): ""
let colorNameParser = str "red" <|> str "blue" <|> str "green" // "red|blue|green"
let colorCountParser = tuple2 (pint32 .>> str " ") colorNameParser // "(n) red|blue|green"
let colorCountListParser = sepBy colorCountParser (str ", ")
let drawListParser = sepBy colorCountListParser (str "; ")
let gameParser = tuple2 gameNumberParser drawListParser

let parseRow (s: string) =
    match run gameParser s with
    | Success (result, _, _) -> result
    | Failure (message, _, _) -> failwith message

// how do I write a function that explitly takes the 

let testRow = "Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green"
let parsedRow = parseRow testRow
printfn $"Test game parser: {parsedRow}"

Test game parser: (5, [[(6, red); (1, blue); (3, green)]; [(2, blue); (1, red); (2, green)]])


In [116]:
type CubeCount = private CubeCount of int array

module CubeCount =
    let create r g b = CubeCount [| r ; g ; b |]

    let value cubeCount = 
        match cubeCount with CubeCount arr -> arr

    let couldBeDrawnFrom bagCount drawCount =
        (value bagCount, value drawCount)
        ||> Array.forall2 (>=)

    let private countForColorFromTuples color xs =
        xs
        |> List.tryFind (fun (_, c) -> c = color)
        |> Option.map fst
        |> Option.defaultValue 0

    let fromTupleList (xs : (int * string) list) =
        CubeCount [|
            xs |> countForColorFromTuples "red"
            xs |> countForColorFromTuples "green"
            xs |> countForColorFromTuples "blue"
            |]

    let maximalArray<'a> =
        Array.map2 max

    let maximalDraw (draws: CubeCount list) =
        draws
        |> List.reduce (fun (CubeCount a) (CubeCount b) ->
            CubeCount (maximalArray a b))

    let product cubeCount =
        match cubeCount with
        | CubeCount [| r; g; b |] -> r * g * b


printfn $"""{CubeCount.fromTupleList [ (3, "blue"); (56, "red") ]}"""



CubeCount [|56; 0; 3|]


In [117]:
let bagConfig = CubeCount.create 12 13 14

display "Part One Answer:"

loadFile data
|> Array.map parseRow
|> Array.map (fun (g, c) -> (g, c |> List.map CubeCount.fromTupleList))
|> Array.filter (fun (g, draws) ->
    draws |> List.forall (fun draw -> draw |> CubeCount.couldBeDrawnFrom bagConfig)) 
|> Array.sumBy fst



Part One Answer:

2285

In [118]:
display "Part Two Answer:"

loadFile data
|> Array.map (parseRow >> snd >> List.map CubeCount.fromTupleList )
|> Array.map CubeCount.maximalDraw
|> Array.map CubeCount.product
|> Array.sum





Part Two Answer:

77021